# 예보 데이터 불러오기
[인증키 확인](https://data.go.kr/iim/api/selectAPIAcountView.do)

In [ ]:
! pip install xmltodict

In [2]:
import pandas as pd
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict

In [3]:
# API
ServiceKey = input('encoded api key를 입력해주세요 > ')

encoded api key를 입력해주세요 > ㅁ


In [ ]:
# ulsan 좌표
x = 102
y = 83

url_base = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtFcst'
key = ServiceKey
queryParams = urlencode({
    # quote_plus('ServiceKey'): key,
    quote_plus('numOfRows'): '113',      # 불러오고 싶은만큼 가져오기
    quote_plus('dataType'): 'XML',
    quote_plus('base_date'): '20210530', # 예보 발표 날짜
    quote_plus('base_time'): '1000',     # 예보 발표 시간
    quote_plus('nx'): str(x),
    quote_plus('ny'): str(y),
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
results_json = xmltodict.parse(results)

data = json.loads(json.dumps(results_json))

if 'ERROR' in data['response']['header']['resultMsg']:
    print('Fail.')
else:
    print('Success!')

In [ ]:
# data 대강 확인하기
data

In [ ]:
logs = data['response']['body']['items']['item']
df = pd.DataFrame()

for log in logs:
    s_tmp = pd.Series(log)
    df_tmp = pd.DataFrame(s_tmp)
    df = pd.concat([df, df_tmp.T])

def fcst_time(time):
    temp = f'{time[:4]}-{time[4:6]}-{time[6:8]} {time[9:11]}:{time[11:]}'
    datetime = pd.to_datetime(temp)
    return datetime

df['time'] = df[['fcstDate', 'fcstTime']].apply(lambda t: ' '.join(t), axis=1)
df['time'] = df['time'].apply(fcst_time)

fcst_df = pd.DataFrame(columns=['time', 'fcst_temp', 'fcst_hum', 'fcst_wspeed',
                                'fcst_wdirect', 'fcst_cloud'])

df